In [89]:
:opt no-lint

# Chapter 12. Signaling adversity
## 12.1 Signaling adversity
* Datatypes to defend against wrong inputs
* Maybe
* Either
* Higher-kindedness
* Anamorphisms

## 12.2 How I learned to stop worrying and love Nothing
We can wrap output inside a `Maybe` to handle invalid inputs.
Smart constructors for datatypes
The same technique can be applied to constructor by defining a function that return a `Maybe` of a type.
## 12.3 Bleathing either
The `Either` sum datatype is useful to signal also _how_ an input is invalid. The `Left` data contructor is conventionally the error constructor becuase it has a stop semantic into the `Functor` instance.
## 12.4 Kinds, a thousand stars in your types
* The `::` syntax is used for both kind signatures and type signatures.
* The `Maybe` and `Either` datatypes have non constant type constructors.
* A **lifted** type is any that can be inhabited by _bottom_, while **unlifted** types aren't.
* Kind `*` is the kind of all standard lifted types
* Kind `#` is the kind of all unlifted types. I.e: native machine types, raw pointer, `newtype` types
* Data constructors are functions. They can be recongnized with `:info`.

## 12.5 Chapter Exercises
### Determine the kinds
1. Given `id :: a -> a` what is the kind of `a`? `*`
2. `r :: a -> f a` What are the kinds of `a` and `f`? `a` is `*`,  `f` is `* -> *` (because the kind of `->` is `* -> * -> *`)

### String processing
1. Write a recursive function named `replaceThe` which takes a text/string, breaks it into words and replaces each instance of “the” with “a”. It’s intended only to replace exactly the word “the”. `notThe` is a suggested helper function for accomplishing this.

In [90]:
import Data.Maybe

notThe :: String -> Maybe String
notThe "the" = Nothing
notThe x = Just x

replaceThe :: String -> String
replaceThe = unwords . go . words
  where
    go [] = []
    go (w:ws) =
      case notThe w of
        Just w -> w : go ws
        Nothing -> "a" : go ws

2. Write a recursive function that takes a text/string, breaks it into words, and counts the number of instances of ”the” followed by a vowel-initial word.

In [91]:
countTheBeforeVowel :: String -> Integer
countTheBeforeVowel = go . words 

go [] = 0
go ("the" : (c : cs) : ws) 
  | isVowel c = 1 + go ws
  | otherwise = go ws
go (_:ws) = go ws

isVowel = flip elem "aeiouyAEIOUY"

3. Return the number of letters that are vowels in a word.

In [92]:
countVowels :: String -> Integer
countVowels = toInteger . length . filter isVowel

isVowel = flip elem "aeiouyAEIOUY"

### Validate the word
Use the `Maybe` type to write a function that counts the number of vowels in a string and the number of consonants.

In [93]:
newtype Word' = Word' String deriving (Eq, Show)

vowels = "aeiou"

mkWord :: String -> Maybe Word'
mkWord x
  | countConsonants x >= countVowels x = Just (Word' x)
  | otherwise = Nothing
    where  
      countConsonants =  toInteger . length . filter (not . isVowel)

### It’s only Natural
You’ll be presented with a datatype to represent the natural numbers. The only values representable with the naturals are whole numbers from zero to infinity. Your task will be to implement functions to convert Naturals to Integers and Integers to Naturals.

In [94]:
data Nat = Zero | Succ Nat deriving (Eq, Show)

natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ n) = 1 + natToInteger n

integerToNat :: Integer -> Maybe Nat
integerToNat n
  | n < 0 = Nothing
  | otherwise = Just $ Succ x
    where
      Just x = integerToNat $ n - 1

### Small library for `Maybe`
Write the following functions. This may take some time.
1. Simple boolean checks for `Maybe` values.

In [95]:
isJust :: Maybe a -> Bool
isJust (Just _) = True
isJust Nothing = False

isNothing :: Maybe a -> Bool
isNothing = not . isJust

2. The following is the `Maybe` catamorphism. You can turn a `Maybe` value into anything else with this.

In [96]:
mayybee :: b -> (a -> b) -> Maybe a -> b
mayybee _ f (Just x) = f x
mayybee x _ Nothing = x

3. In case you just want to provide a fallback value.

In [97]:
fromMaybe :: a -> Maybe a -> a
fromMaybe = flip mayybee id

4. Converting between `List` and `Maybe`.

In [98]:
listToMaybe :: [a] -> Maybe a
listToMaybe [] = Nothing
listToMaybe (x:_) = Just x

maybeToList :: Maybe a -> [a]
maybeToList = mayybee [] (: [])

5. For when we want to drop the `Nothing` values from our list.

In [99]:
catMaybes :: [Maybe a] -> [a]
catMaybes = concatMap maybeToList

6. You’ll see this called “sequence” later.

In [100]:
flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe = foldr go (Just [])
  where 
    go (Just x) (Just xs) = Just (x : xs)
    go _        _          = Nothing

### Small library for `Etiher`
1. Try to eventually arrive at a solution that uses `foldr`, even if earlier versions don’t use `foldr`.

In [101]:
lefts' :: [Either a b] -> [a]
lefts' = foldr go []
  where 
    go :: Either a b -> [a] -> [a]
    go (Left x) xs = x : xs
    go (Right _) xs = xs

2. Same as the last one. Use `foldr` eventually.

In [102]:
rights' :: [Either a b] -> [b]
rights' = foldr go []
  where 
    go :: Either a b -> [b] -> [b]
    go (Left _) xs = xs
    go (Right x) xs = x:xs

3.

In [103]:
partitionEithers' :: [Either a b] -> ([a], [b])
partitionEithers' = foldr go ([], [])
  where 
    go (Left  a) (as, bs) = (a:as, bs)
    go (Right b) (as, bs) = (as, b:bs)

4.

In [104]:
eitherMaybe' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe' f (Right x) =  Just $ f x
eitherMaybe' _ (Left _) = Nothing

5. This is a general catamorphism for `Either` values.

In [105]:
either' :: (a -> c) -> (b -> c) -> Either a b -> c
either' _ f (Right x) =  f x
either' f _ (Left x) = f x

6. Same as before, but use the `either'` function you just wrote.

In [106]:
eitherMaybe'' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe'' f = either' (const Nothing) (Just . f)

### Unfolds
The dual of catamorphism is _anamorphism_

1. Write the function `iterate'` using direct recursion.

In [107]:
iterate' :: (a -> a) -> a -> [a]
iterate' f x = x : iterate' f (f x)

2. Write the function `unfoldr'` using direct recursion.

In [108]:
unfoldr' :: (b -> Maybe (a, b)) -> b -> [a]
unfoldr' f state = case f state of
  Just (a, b) -> a : unfoldr' f b
  Nothing -> []

3. Rewrite `iterate'` into `betterIterate` using `unfoldr'`.

In [109]:
betterIterate :: (a -> a) -> a -> [a]
betterIterate f = unfoldr' f'
  where
    f' x = Just (x, f x)

### Finally something other than a list!

In [110]:
data BinaryTree a = Leaf | Node (BinaryTree a) a (BinaryTree a) deriving (Eq, Ord, Show)

1. Write `unfold` for `BinaryTree`.

In [111]:
unfoldTree :: (a -> Maybe (a, b, a)) -> a -> BinaryTree b
unfoldTree f xs = case f xs of
  Just (a, b, a') -> Node (unfoldTree f a) b (unfoldTree f a')
  Nothing -> Leaf

2. Make a tree builder. Using the `unfoldTree` function you’ve made for `BinaryTree`, write the following function:

In [112]:
treeBuild :: Integer -> BinaryTree Integer
treeBuild n = unfoldTree f 0
  where
    f x
      | x < n    = Just (x + 1, x, x + 1)
      | otherwise = Nothing

## 12.6 Definitions
1. A higher-kinded type type is any type whose kind has a function arrow in it and which can be described as a type constructor. This is not to be confused with higher kinded _polymorphism_